In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.optim.lr_scheduler import StepLR

# Load dataset

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root='./data', download=True, train=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', download=True, train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)


In [ ]:
class CustomConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, stride, padding):
        super(CustomConvLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, dilation=dilation, stride=stride, padding=padding)

    def forward(self, x):
        out = self.conv(x)
        windows = torch.nn.functional.unfold(x, kernel_size=self.conv.kernel_size, dilation=self.conv.dilation[0], stride=self.conv.stride[0], padding=self.conv.padding[0])
        windows = windows.reshape(x.shape[0], 1, self.conv.in_channels * self.conv.kernel_size[0]**2, out.shape[-2], out.shape[-1])
        return out, windows


In [ ]:
class CustomMaxPoolLayer(nn.Module):
    def __init__(self, kernel_size, dilation, stride):
        super(CustomMaxPoolLayer, self).__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=kernel_size, dilation=dilation, stride=stride)

    def forward(self, x):
        out = self.max_pool(x)
        return out

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = CustomConvLayer(in_channels=1, out_channels=32, kernel_size=3, dilation=2, stride=1, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, dilation=2, stride=2)
        self.conv2 = CustomConvLayer(in_channels=32, out_channels=64, kernel_size=3, dilation=2, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, dilation=2, stride=2)

        self.fc1 = nn.Linear(576, 512) 
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)  

    def forward(self, x):
        x, _ = self.conv1(x)
        x = F.relu(x)
        x = self.maxpool1(x)

        x, _ = self.conv2(x)
        x = F.relu(x)
        x = self.maxpool2(x)

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN()
model.to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)


dummy_input = torch.randn(1, 1, 28, 28).to('cuda')


output = model(dummy_input)

num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss}")

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        accuracy = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy * 100:.2f}%")
